In [144]:
import pandas as pd
import numpy as np

In [146]:
signals = pd.read_csv('data/signals_mispricing.csv')

In [147]:
market_data = pd.read_csv('data/clean/SPY_2023_eod.csv')

In [150]:
signals.head()

,QUOTE_DATE,EXPIRE_DATE,STRIKE,DTE,moneyness,mid_iv,atm_iv,skew,term_spread,trade_type,confidence
0,2023-08-01,2023-08-01,370.0,0.0,0.810604,2.483815,0.368407,-5.294118,0.019383,SHORT,0.999911
1,2023-08-01,2023-08-01,380.0,0.0,0.832512,2.221290,0.368407,-5.294118,0.019383,SHORT,0.999911
2,2023-08-01,2023-08-01,388.0,0.0,0.850038,2.020245,0.368407,-5.294118,0.019383,SHORT,0.999912
3,2023-08-01,2023-08-01,390.0,0.0,0.854420,1.948275,0.368407,-5.294118,0.019383,SHORT,0.999877
4,2023-08-01,2023-08-01,395.0,0.0,0.865374,1.821315,0.368407,-5.294118,0.019383,SHORT,0.999775


In [151]:
market_data.head()

,QUOTE_DATE,QUOTE_TIME_HOURS,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,C_BID,C_ASK,C_IV,C_DELTA,...,P_ASK,P_IV,P_DELTA,P_GAMMA,P_THETA,P_VEGA,P_RHO,P_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT
0,2023-08-01,16.0,456.45,2023-08-01,0.0,370.0,86.49,86.58,3.61938,0.97443,...,0.01,1.34825,-0.00084,0.00003,-0.00491,0.00060,0.00000,0.0,86.4,0.189
1,2023-08-01,16.0,456.45,2023-08-01,0.0,380.0,76.49,76.58,3.26940,0.97349,...,0.01,1.17318,-0.00126,0.00015,-0.00504,0.00083,0.00000,0.0,76.4,0.167
2,2023-08-01,16.0,456.45,2023-08-01,0.0,388.0,68.49,68.59,3.00529,0.97023,...,0.01,1.03520,-0.00132,0.00014,-0.00463,0.00069,-0.00022,0.0,68.4,0.150
3,2023-08-01,16.0,456.45,2023-08-01,0.0,390.0,65.96,67.04,2.89604,0.97611,...,0.01,1.00051,-0.00046,0.00013,-0.00491,0.00050,0.00000,0.0,66.4,0.146
4,2023-08-01,16.0,456.45,2023-08-01,0.0,395.0,60.96,62.03,2.72683,0.97421,...,0.01,0.91580,-0.00140,0.00018,-0.00473,0.00034,-0.00019,0.0,61.4,0.135


In [175]:
market_data.columns

Index(['QUOTE_DATE', 'QUOTE_TIME_HOURS', 'UNDERLYING_LAST', 'EXPIRE_DATE',
       'DTE', 'STRIKE', 'C_BID', 'C_ASK', 'C_IV', 'C_DELTA', 'C_GAMMA',
       'C_THETA', 'C_VEGA', 'C_RHO', 'C_VOLUME', 'P_BID', 'P_ASK', 'P_IV',
       'P_DELTA', 'P_GAMMA', 'P_THETA', 'P_VEGA', 'P_RHO', 'P_VOLUME',
       'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT'],
      dtype='object')

In [176]:
signals.columns

Index(['QUOTE_DATE', 'EXPIRE_DATE', 'STRIKE', 'DTE', 'moneyness', 'mid_iv',
       'atm_iv', 'skew', 'term_spread', 'trade_type', 'confidence'],
      dtype='object')

In [161]:
market_data['EXPIRE_DATE']

0         2023-08-01
1         2023-08-01
2         2023-08-01
3         2023-08-01
4         2023-08-01
             ...    
578738    2026-01-16
578739    2026-01-16
578740    2026-01-16
578741    2026-01-16
578742    2026-01-16
Name: EXPIRE_DATE, Length: 578743, dtype: object

In [172]:
keys = ['QUOTE_DATE', 'STRIKE', 'EXPIRE_DATE']
merged_df = pd.merge(signals, market_data, on=keys, how='inner')

In [173]:
merged_df.head()

,QUOTE_DATE,EXPIRE_DATE,STRIKE,DTE_x,moneyness,mid_iv,atm_iv,skew,term_spread,trade_type,...,P_ASK,P_IV,P_DELTA,P_GAMMA,P_THETA,P_VEGA,P_RHO,P_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT
0,2023-08-01,2023-08-01,370.0,0.0,0.810604,2.483815,0.368407,-5.294118,0.019383,SHORT,...,0.01,1.34825,-0.00084,0.00003,-0.00491,0.00060,0.00000,0.0,86.4,0.189
1,2023-08-01,2023-08-01,380.0,0.0,0.832512,2.221290,0.368407,-5.294118,0.019383,SHORT,...,0.01,1.17318,-0.00126,0.00015,-0.00504,0.00083,0.00000,0.0,76.4,0.167
2,2023-08-01,2023-08-01,388.0,0.0,0.850038,2.020245,0.368407,-5.294118,0.019383,SHORT,...,0.01,1.03520,-0.00132,0.00014,-0.00463,0.00069,-0.00022,0.0,68.4,0.150
3,2023-08-01,2023-08-01,390.0,0.0,0.854420,1.948275,0.368407,-5.294118,0.019383,SHORT,...,0.01,1.00051,-0.00046,0.00013,-0.00491,0.00050,0.00000,0.0,66.4,0.146
4,2023-08-01,2023-08-01,395.0,0.0,0.865374,1.821315,0.368407,-5.294118,0.019383,SHORT,...,0.01,0.91580,-0.00140,0.00018,-0.00473,0.00034,-0.00019,0.0,61.4,0.135


In [174]:
merged_df.columns

Index(['QUOTE_DATE', 'EXPIRE_DATE', 'STRIKE', 'DTE_x', 'moneyness', 'mid_iv',
       'atm_iv', 'skew', 'term_spread', 'trade_type', 'confidence',
       'QUOTE_TIME_HOURS', 'UNDERLYING_LAST', 'DTE_y', 'C_BID', 'C_ASK',
       'C_IV', 'C_DELTA', 'C_GAMMA', 'C_THETA', 'C_VEGA', 'C_RHO', 'C_VOLUME',
       'P_BID', 'P_ASK', 'P_IV', 'P_DELTA', 'P_GAMMA', 'P_THETA', 'P_VEGA',
       'P_RHO', 'P_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT'],
      dtype='object')

In [177]:
trade_data = merged_df[['QUOTE_DATE', 'EXPIRE_DATE', 'STRIKE','trade_type', 'confidence']]

In [178]:
trade_data['entry_price'] = merged_df[['P_BID'] + ['P_ASK']].mean(axis=1)

/var/folders/d1/8stldbwd0c5f13jlmcfpc6x40000gn/T/ipykernel_74031/1351693746.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade_data['entry_price'] = merged_df[['P_BID'] + ['P_ASK']].mean(axis=1)


In [179]:
trade_data.head()

,QUOTE_DATE,EXPIRE_DATE,STRIKE,trade_type,confidence,entry_price
0,2023-08-01,2023-08-01,370.0,SHORT,0.999911,0.005
1,2023-08-01,2023-08-01,380.0,SHORT,0.999911,0.005
2,2023-08-01,2023-08-01,388.0,SHORT,0.999912,0.005
3,2023-08-01,2023-08-01,390.0,SHORT,0.999877,0.005
4,2023-08-01,2023-08-01,395.0,SHORT,0.999775,0.005


In [180]:
class Trade:
    
    def __init__(self, entry_date, exit_date, strike, trade_type, entry_price, confidence, quantity=1):
        self.entry_date = entry_date
        self.exit_date = exit_date
        self.strike = strike
        self.trade_type = trade_type
        self.entry_price = entry_price
        self.confidence = confidence
        self.quantity = quantity
        
    def is_active(self, date: str):
        return self.entry_date <= date <= self.exit_date

In [181]:
class StoreTrades:
    def __init__(self):
        self.trades = []
        
    def add_trade(self, trade: Trade):
        self.trades.append(trade)
        
    def get_trades(self):
        return self.trades
        

In [182]:
trades = StoreTrades()
trades.add_trade(Trade(trade_data['QUOTE_DATE'], trade_data['EXPIRE_DATE'], trade_data['STRIKE'], trade_data['trade_type'], trade_data['entry_price'], trade_data['confidence']))


In [186]:
trades = StoreTrades()

for _, row in trade_data.iterrows():
    trade = Trade(
        entry_date=row['QUOTE_DATE'],
        exit_date=row['EXPIRE_DATE'],
        strike=row['STRIKE'],
        trade_type=row['trade_type'],
        entry_price=row['entry_price'],
        confidence=row['confidence'],
        quantity=1  # or dynamic later
    )
    trades.add_trade(trade)


In [187]:
trades.get_trades()[0].entry_date

'2023-08-01'